In [60]:
import numpy
import time

from discrete_fuzzy_operators.base.operators.binary_operators.discrete.suboperators.fuzzy_discrete_aggregation_operator import DiscreteAggregationBinaryOperator
from tqdm.notebook import tqdm
from typing import Dict, List, Tuple

	 Computing all possible operators.: 4025724406278it [01:42, 39397623915.43it/s] 


In [47]:
def generate_row_compensation_space(previous_row: List,
                                    admissible_range: Tuple[int, int],
                                    max_vector_size: int,
                                    row_position: int,
                                    recursive_step: int = 0,
                                    row_list: List = None) -> List[int]:
    """
    A generator which builds are possible vectors of a certain size (max_vector_size) that are increasing, where its
    entries are taken from a range (admissible_range, representing a discrete interval [a,b]) and component-wise are 
    bigger than a given vector (previous_row).

    Args:
        previous_row: A list of integers, representing the vector whose entries are the lower-bounds of the generated vectors.
        admissible_range: A tuple of two integers, representing the interval from which the candidate values will be taken.
        max_vector_size: An integer, representing the size of the vector to be generated.
        recursive_step: An integer, representing the number of recursive calls. It is used as the stop criterion.
        row_list: A list of integers, representing the generated vector.

    Returns:
        A list of integers, representing the vector with size max_vector_size and which entries are in increasing order.
    """
    assert max_vector_size <= len(previous_row), "Number of entries of the generated vector is greater than the size of the bound vector. Expected to be less than or equal to its size."
    
    min_value = admissible_range[0]
    max_value = admissible_range[1]
    
    if row_list is None:
        row_list = []

    if recursive_step == max_vector_size:
        yield row_list
    else:
        minimum_row_value = min_value

        if len(previous_row) > 0:
            minimum_row_value = max(min_value, previous_row[recursive_step])

        for i in range(minimum_row_value, max_value + 1):
            if min(recursive_step,row_position) <= i <= max(recursive_step,row_position):
                new_row = row_list.copy()
                new_row.append(i)
                yield from generate_row_compensation_space(previous_row=previous_row,
                                                           admissible_range=(i,max_value),
                                                           max_vector_size=max_vector_size,
                                                           row_position=row_position,
                                                           recursive_step=recursive_step+1,
                                                           row_list=new_row)
            else:
                continue
                
def generate_recursive_compensation_space(n: int, e: int, row: int, previous_rows: List):
    if row == n:
        yield previous_rows
    else:
        previous_boundary = previous_rows[len(previous_rows)-1]
        for next_row in generate_row_compensation_space(previous_row=previous_boundary, 
                                                        admissible_range=(0,row+1), 
                                                        max_vector_size=e,
                                                        row_position=row+1):
            if (row+1) == n:
                if next_row[0] == 0 or next_row[0] == n: #Every discrete uninorms satisfies that U(n,0)=0 or U(n,0)=n.
                    next_iteration_rows = previous_rows.copy()
                    next_iteration_rows.append(next_row)
                    yield from generate_recursive_compensation_space(n=n, e=e, row=row+1, previous_rows=next_iteration_rows)
                else:
                    continue
            else:
                next_iteration_rows = previous_rows.copy()
                next_iteration_rows.append(next_row)
                yield from generate_recursive_compensation_space(n=n, e=e, row=row+1, previous_rows=next_iteration_rows)
                
def generate_compensation_spaces(n: int, e: int):
    boundary = [i for i in range(0, e)]
    row = e+1
    
    for seed in generate_row_compensation_space(previous_row=boundary, 
                                                admissible_range=(0,row), 
                                                max_vector_size=e,
                                                row_position=row):

        for compensation_space in generate_recursive_compensation_space(n=n, e=e, row=row, previous_rows=[seed]):
            yield numpy.flipud(numpy.array(compensation_space))

## Prova I

Provam de generar tots els possibles espais de compensació i, com que coneixem el nombre de t-normes i t-conormes, podem intentar proposar una expressió.

In [ ]:
def get_number_spaces(n: int, e: int):
    number_spaces_n = 0
    for e in range(1, (n-1)+1):
        for space in generate_compensation_spaces(n=n, e=e):
            number_spaces_n += 1
    return number_spaces_n

number_tnorms = {1:1, 2:2, 3:6, 4:22, 5:94, 6:451, 7:2386, 8:13775, 9:86417, 10:590489, 11:4446029, 12:37869449, 13:382549464}
number_uninorms = {}

for n in range(2,6+1):
    uni = 2*number_tnorms[n]
    
    for e in range(1,(n-1)+1):
        uni += number_tnorms[e]*number_tnorms[n-e]*get_number_spaces(n=n, e=e)
    number_uninorms[n] = uni
    
print(number_uninorms)

Provem-ho amb un valor concret, amb $n=3$, per veure si la prova anterior és correcta.

In [ ]:
from discrete_fuzzy_operators.base.operators.binary_operators.discrete.suboperators.fuzzy_discrete_aggregation_operator import DiscreteAggregationBinaryOperator

template = numpy.array([[0,0,0,0], [0,1,2,3], [0,2,0,3], [0,3,3,3]])

num = 0
for x1 in range(0, 2+1):
    for x2 in range(x1, 3+1):
        for x3 in range(2,3+1):
            another = template.copy()
            another[0,2] = x1
            another[2,0] = x1
            another[0,3] = x2
            another[3,0] = x2
            another[2,2] = x3
            
            operator = DiscreteAggregationBinaryOperator(n=3, operator_matrix=another)
            if operator.is_associative():
                num += 1
print(num)

In [ ]:
template = numpy.array([[0,0,0,0], [0,0,1,0], [0,1,2,3], [0,0,3,3]])

num = 0
for x1 in range(0, 1+1):
    for x2 in range(0, 3+1):
        for x3 in range(max(1,x2), 3+1):
            another = template.copy()
            another[1,1] = x1
            another[0,3] = x2
            another[3,0] = x2
            another[1,3] = x3
            another[3,1] = x3
            
            operator = DiscreteAggregationBinaryOperator(n=3, operator_matrix=another)
            if operator.is_associative():
                print(numpy.flipud(another))
                num += 1
print(num)

Sembla que no és correcte. Per aquest valor concret, tenim $6+5+5+6=22$ uninormes possibles a $L_3$, mentres que amb la prova anterior ha sortit $84$. Com que al primer experiment no es comprova l'associativitat, segurament aquest és el motiu pel qual surten valors tan grans. A la següent prova, generem-les totes i mirem si són operadors associatius.

## Prova II

In [18]:
discrete_dataset_path = r"C:\Users\Usuario\OneDrive - Universitat de les Illes Balears\UIB\Tesi\Experiments\DiscreteDataset"

available_tnorms = {}
available_tconorms = {}

for n in range(1, 8+1):
    available_tnorms[n]   = numpy.load(discrete_dataset_path+rf"\N={n}\tnorms.npy", allow_pickle=True)
    available_tconorms[n] = numpy.load(discrete_dataset_path+rf"\N={n}\tconorms.npy", allow_pickle=True)

In [61]:
def check_associativitiy(operator: numpy.ndarray, n: int) -> bool:
    for x in range(0, n+1):
        for y in range(0, n+1):
            for z in range(0, n+1):
                v1 = operator[y,z]
                v2 = operator[x,y]
                
                if operator[x, v1] != operator[v2, z]:
                    return False
    return True

def generate_uninorms(n: int, available_tnorms: Dict[int, List[numpy.array]], available_tconorms: Dict[int, List[numpy.array]], verbose: bool = False) -> Tuple[List[numpy.array], float]:
    verboseprint = print if verbose else lambda *a, **k: None
    
    available_uninorms = []
    start_time = time.time()

    for e in range(0, n+1):
        num_e = 0
        if e == 0:
            verboseprint(f"---------- WORKING WITH e={e} ----------")
            verboseprint(len(list(available_tnorms[n])))
            available_uninorms = available_uninorms+list(available_tnorms[n])
        elif e == n:
            verboseprint(f"---------- WORKING WITH e={e} ----------")
            verboseprint(len(list(available_tconorms[n])))
            available_uninorms = available_uninorms+list(available_tconorms[n])
        else:
            verboseprint(f"---------- WORKING WITH e={e} ----------")
            for underlying_tnorm in available_tnorms[e]:
                for underlying_tconorm in available_tconorms[n-e]:
                    for compensation_space in generate_compensation_spaces(n=n, e=e):

                        compensation_space_mod = numpy.flipud(compensation_space)

                        uninorm_candidate = (-1)*numpy.ones((n+1, n+1), dtype=int)
                        uninorm_candidate[0:(e+1), 0:(e+1)] = underlying_tnorm
                        uninorm_candidate[e:(n+1), e:(n+1)] = e+underlying_tconorm
                        uninorm_candidate[(e+1):(n+1), 0:e] = compensation_space_mod
                        uninorm_candidate[0:e, (e+1):(n+1)] = numpy.transpose(compensation_space_mod)

                        operator = DiscreteAggregationBinaryOperator(n=n, operator_matrix=uninorm_candidate)
                        if operator.is_associative():
                            available_uninorms.append(uninorm_candidate)
                            num_e += 1
            verboseprint(num_e)
    elapsed_time = time.time()-start_time
    
    return available_uninorms, elapsed_time

def count_uninorms(n: int, available_tnorms: Dict[int, List[numpy.array]], available_tconorms: Dict[int, List[numpy.array]], verbose: bool = False) -> Tuple[List[numpy.array], float]:
    verboseprint = print if verbose else lambda *a, **k: None
    
    available_uninorms = 0
    start_time = time.time()
    
    if n%2 == 0:
        for e in range(0, int(n/2+1)):
            if e == 0:
                verboseprint(f"---------- WORKING WITH e={e} AND e={n} ----------")
                verboseprint(2*len(list(available_tnorms[n])))
                available_uninorms = available_uninorms+2*len(list(available_tnorms[n]))
            else:
                available_uninorms_e = 0
                factor = 2
                if e == (int(n/2+1)-1):
                    factor = 1
                
                verboseprint(f"---------- WORKING WITH e={e} AND e={n-e} ----------")
                compensation_spaces = [space for space in generate_compensation_spaces(n=n, e=e)]
                progress_bar = tqdm(desc="\t Computing all possible operators.", total=len(compensation_spaces)*len(available_tnorms[e]*len(available_tconorms[n-e])))
                done_cases = 0
                
                for compensation_space in compensation_spaces:
                    for underlying_tnorm in available_tnorms[e]:
                        for underlying_tconorm in available_tconorms[n-e]:
                            compensation_space_mod = numpy.flipud(compensation_space)

                            uninorm_candidate = (-1)*numpy.ones((n+1, n+1), dtype=int)
                            uninorm_candidate[0:(e+1), 0:(e+1)] = underlying_tnorm
                            uninorm_candidate[e:(n+1), e:(n+1)] = e+underlying_tconorm
                            uninorm_candidate[(e+1):(n+1), 0:e] = compensation_space_mod
                            uninorm_candidate[0:e, (e+1):(n+1)] = numpy.transpose(compensation_space_mod)

                            if check_associativitiy(operator=uninorm_candidate, n=n):
                                available_uninorms_e += 1
                            
                            done_cases += 1
                            progress_bar.update(1)
                
                progress_bar.close()
                verboseprint(factor*available_uninorms_e)
                available_uninorms += factor*available_uninorms_e
    else:
        for e in range(0, int((n+1)/2)):
            if e == 0:
                verboseprint(f"---------- WORKING WITH e={e} AND e={n} ----------")
                verboseprint(2*len(list(available_tnorms[n])))
                available_uninorms = available_uninorms+2*len(list(available_tnorms[n]))
            else:
                available_uninorms_e = 0
                factor = 2
                
                verboseprint(f"---------- WORKING WITH e={e} AND e={n-e} ----------")
                compensation_spaces = [space for space in generate_compensation_spaces(n=n, e=e)]
                progress_bar = tqdm(desc="\t Computing all possible operators.", total=len(compensation_spaces)*len(available_tnorms[e]*len(available_tconorms[n-e])))
                done_cases = 0
                
                for compensation_space in compensation_spaces:
                    for underlying_tnorm in available_tnorms[e]:
                        for underlying_tconorm in available_tconorms[n-e]:
                            compensation_space_mod = numpy.flipud(compensation_space)

                            uninorm_candidate = (-1)*numpy.ones((n+1, n+1), dtype=int)
                            uninorm_candidate[0:(e+1), 0:(e+1)] = underlying_tnorm
                            uninorm_candidate[e:(n+1), e:(n+1)] = e+underlying_tconorm
                            uninorm_candidate[(e+1):(n+1), 0:e] = compensation_space_mod
                            uninorm_candidate[0:e, (e+1):(n+1)] = numpy.transpose(compensation_space_mod)

                            if check_associativitiy(operator=uninorm_candidate, n=n):
                                available_uninorms_e += 1
                                
                            done_cases += 1
                            progress_bar.update(1)
                
                progress_bar.close()
                verboseprint(factor*available_uninorms_e)
                available_uninorms += factor*available_uninorms_e
    
    elapsed_time = time.time()-start_time
    
    return available_uninorms, elapsed_time

In [15]:
uninorms3, time6 = generate_uninorms(n=6, available_tnorms=available_tnorms, available_tconorms=available_tconorms, verbose=True)
print(len(uninorms3))
print(time6)

---------- WORKING WITH e=0 ----------
451
---------- WORKING WITH e=1 ----------
308
---------- WORKING WITH e=2 ----------
217
---------- WORKING WITH e=3 ----------
194
---------- WORKING WITH e=4 ----------
217
---------- WORKING WITH e=5 ----------
308
---------- WORKING WITH e=6 ----------
451
2146
228.85136103630066


In [14]:
uni6, time6 = count_uninorms(n=6, available_tnorms=available_tnorms, available_tconorms=available_tconorms, verbose=True)
print(uni6)
print(time6)

---------- WORKING WITH e=0 AND e=6 ----------
902
---------- WORKING WITH e=1 AND e=5 ----------
616
---------- WORKING WITH e=2 AND e=4 ----------
434
---------- WORKING WITH e=3 AND e=3 ----------
194
2146
16.159329652786255


In [52]:
uni7, time7 = count_uninorms(n=7, available_tnorms=available_tnorms, available_tconorms=available_tconorms, verbose=True)

---------- WORKING WITH e=0 AND e=7 ----------
4772
---------- WORKING WITH e=1 AND e=6 ----------
3076
---------- WORKING WITH e=2 AND e=5 ----------
2068
---------- WORKING WITH e=3 AND e=4 ----------
1718


In [53]:
print(uni7)
print(time7)

11634
93.15575742721558


In [54]:
uni8, time8 = count_uninorms(n=8, available_tnorms=available_tnorms, available_tconorms=available_tconorms, verbose=True)
print(uni8)
print(time8)

---------- WORKING WITH e=0 AND e=8 ----------
27550
---------- WORKING WITH e=1 AND e=7 ----------
16708
---------- WORKING WITH e=2 AND e=6 ----------
10774
---------- WORKING WITH e=3 AND e=5 ----------
8504
---------- WORKING WITH e=4 AND e=4 ----------
3936
67472
21002.47544813156
